<a href="https://colab.research.google.com/github/samanthasu12/Contentstack_Internship/blob/main/L%26DDataCalculation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Welcome to my L&D Automation Project!**
----
Here are the steps in which you need to take to automate the data.
1.  Make sure that all the sheets are completely blank except for the column titles.
2. Insert the LearnUpon report into the Employee Data sheet. **NOTE:** The report must contain the following columns: First Name,	Last Name,	email,	enabled,	expires,	Course name,	course ref. code,	enrolled,	source,	due date,	started,	Last Accessed,	Last Accessed Module,	completed,	Pass Mark,	Score,	Status,	Can re-attempt?,	Percentage Complete,	Expiration date,	Groups,	Started, Date/Time,	Last Accessed Date/Time,	and Completed Date/Time.
3. Insert the UKG report into the People Ops Data sheet.  **NOTE:** The report must contain the following columns: Employee Number,	Employee Name, (Last Suffix, First MI),	Employee Email,	Employment Status Code,	Employment Type,	Job,	Supervisor Name (First MI Last Suffix),	Org Level 1,	Org Level 2,	Org Level 3,	Org Level 4, and	Location
4. Make sure the sheet names in the google sheet are the following: Course A, Employee Data, and People Ops Data.
5. After you've completed your checks, please click Runtime (located at the very top of the colab) and Run All. It will then ask you to connect the google colab to your own google account, and you just need to sign in.
6. Please give it around 6 minutes to fully automate. (Might be longer depending on how big the report is)


In [ ]:
from google.colab import auth
import pandas as pd
import numpy as np
auth.authenticate_user()
import gspread
from google.auth import default
from oauth2client.client import GoogleCredentials
from oauth2client.service_account import ServiceAccountCredentials
from google.colab import drive
from google.colab import auth
from IPython.display import clear_output
creds, _ = default()
drive.mount('/content/drive/')
creds, _ = default()
gc = gspread.authorize(creds)
drive.mount("mnt")
!pip install --upgrade gspread
import gspread
from datetime import date

sheet_id='1zBvZf07H6HrQNsCRNs2-d22T31KnHi65m5iZbzLgxrY'
#the sheet_id is the "key" to unlocking the spreadsheet that I am importing/exporting onto/from (which in this case is the TA Master Spreadsheet).
#The key is found after /d/ and before /edit in https://docs.google.com/spreadsheets/d/1qZh3oDeFjUqQu8RecKGn-Wb9BE3m0iI3OYab1dBg52I/edit#gid=317505658

sh = gc.open_by_key(sheet_id)
sh.worksheets()

!pip install pandasql
import sqlite3
clear_output()


Read the code block below to sync up the functions.

In [ ]:
#functions
def WorksheetLoading(Name):
  Data_Worksheet=sh.worksheet(Name)
  Data_Worksheet=Data_Worksheet.get_all_values()
  headers1 = Data_Worksheet.pop(0)
  AllEmployeeData=pd.DataFrame(Data_Worksheet, columns=headers1)
  return AllEmployeeData

def WorksheetNaming(Name):
  Sheet_Worksheet=sh.worksheet("Course A")
  Sheet_df=pd.DataFrame(Sheet_Worksheet.get_all_records())
  return Sheet_df

def Update(dataframe,worksheet):
  rows_to_update=dataframe[dataframe['Email']=="*"]
  for row in rows_to_update.itertuples(index=True, name='Pandas'):
    dataframe['Employee Name'] = merged_df.sort_values(by='Email')['Employee_Name']
    dataframe['Email'] = merged_df.sort_values(by='Email')['Email']
    dataframe['Manager? (Y/N)'] = merged_df.sort_values(by='Email')['Manager? (Y/N)']
    dataframe['Courses'] = merged_df.sort_values(by='Email')['Course Name']
    dataframe['Role/Title'] = merged_df.sort_values(by='Email')['Job']
    dataframe['Geo / Location'] = merged_df.sort_values(by='Email')['Location']
    dataframe['Reports To:'] = merged_df.sort_values(by='Email')['Supervisor_Name']
    dataframe['Sub Department / Team'] = merged_df.sort_values(by='Email')['Sub_Department']
    dataframe['Department'] = merged_df.sort_values(by='Email')['Department']
    dataframe['Status'] = merged_df.sort_values(by='Email')['Status']
    dataframe['Percentage Complete']= merged_df.sort_values(by='Email')['Percentage Complete']
    dataframe['Time Spent (ONLY COMPLETED)']= merged_df.sort_values(by='Email')['Days Taken']
    dataframe = merged_df.fillna('')
    dataframe['Email'] = dataframe['Email'].astype(str) + dataframe['Email'].ne(dataframe['Email'].shift()).replace({True: ' ', False: ''})
  worksheet.update([dataframe.columns.values.tolist()] + dataframe.values.tolist())

def split_dataframe_by_unique_values(df, column_name):
  unique_values = df[column_name].unique()
  dataframes = []
  for value in unique_values:
      subset_df = df[df[column_name] == value].copy()
      dataframes.append(subset_df)
  return dataframes

The code below loads the LearnUpon Data from the excel (found under the Employee Data sheet)

In [ ]:
EmployeeData_Df=WorksheetLoading('Employee Data')
EmployeeData_Df['Name'] = EmployeeData_Df.apply(lambda row: str(row['Last Name']) +', ' + row['First Name'], axis=1)
EmployeeData_Df['Full_Name']=EmployeeData_Df['Name'].str.title()
EmployeeData_Df
clear_output()


The code below calculates the days taken, whether a person is a manager, and whether a course is completed. **NOTE:** it takes around 5 minutes to load.

In [ ]:
frequency_df=EmployeeData_Df[['email','First Name', 'Last Name', 'Status','Percentage Complete','Course name']]
frequency_df.columns = ['Email', 'First Name','Last Name', 'Status','Percentage Complete','Course Name']
frequency_df = frequency_df.assign(Employee_Name=frequency_df['First Name'].astype(str) +' '+ frequency_df['Last Name'])
frequency_df=frequency_df.drop('First Name', axis=1)
frequency_df=frequency_df.drop('Last Name', axis=1)
for email in frequency_df['Email']:
    count = 'Not Completed'
    for idx, row in EmployeeData_Df.iterrows():
      if 'Managers at Contentstack' in row['Groups'] and email == row['email']:
        frequency_df.at[idx, 'Manager? (Y/N)'] = 'Y'
frequency_df['Manager? (Y/N)'] = frequency_df['Manager? (Y/N)'].fillna('N')
frequency_df=frequency_df.reset_index()
frequency_df.replace({np.nan: 'Not Completed'}, inplace = True)
frequency_df.replace({'': '0 '}, inplace = True)
frequency_df.replace({'Not Completed': ' '}, inplace = True)
frequency_df=frequency_df.rename(columns={"Days Taken": "Days_Taken",
                                          "Manager? (Y/N)": "Manager?"})
frequency_df['Percentage Complete'] = frequency_df['Percentage Complete'].replace('-', '0%')
frequency_df
clear_output()


The code below reads the People Ops Data that I got from UKG (another dataset)

In [ ]:
PeopleOps_Df=WorksheetLoading('People Ops Data')
PeopleOps_Df=PeopleOps_Df.rename(columns={"Employee Email": "email",
                                          "Employee Name (Last Suffix, First MI)": "Full_Name",
                                          "Supervisor Name (First MI Last Suffix)": "Supervisor_Name",
                                          "Org Level 1": "Department",
                                          "Org Level 2": "Sub_Department"})

PeopleOps_Df.head(30)
clear_output()


Run this below to install SQL

In [ ]:
pip install SQLAlchemy==2.0.18


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 31.0 MB/s eta 0:00:00
  Attempting uninstall: SQLAlchemy
    Found existing installation: SQLAlchemy 2.0.20
    Uninstalling SQLAlchemy-2.0.20:
      Successfully uninstalled SQLAlchemy-2.0.20


The code below uses SQL to join the data together

In [ ]:
from sqlalchemy import create_engine

engine = create_engine('sqlite://',
                       echo = False)

EmployeeData_Df.to_sql('AllEmployeeData',
               con = engine)
PeopleOps_Df.to_sql('PeopleOpsData',
               con = engine)

with engine.connect() as conn:
  Connect_df = conn.exec_driver_sql("SELECT PeopleOpsData.email, PeopleOpsData.Full_Name, PeopleOpsData.Job, PeopleOpsData.Supervisor_Name, PeopleOpsData.Department, PeopleOpsData.Sub_Department, PeopleOpsData.Location FROM PeopleOpsData LEFT JOIN AllEmployeeData ON AllEmployeeData.email = PeopleOpsData.email").fetchall()
Connect_df=pd.DataFrame(Connect_df)
Connect_df=Connect_df.rename(columns={"email": "Email"})
Connect_df
clear_output()


The code below uses Python to join additional data together.

In [ ]:
merged_df = pd.merge(frequency_df, Connect_df, on='Email', how='inner')
merged_df=merged_df.drop_duplicates()
merged_df=merged_df.reset_index()
merged_df=merged_df.drop('index',axis=1)
merged_df=merged_df.sort_values('Course Name')
merged_df=merged_df.reset_index()
merged_df=merged_df.drop('index',axis=1)
merged_df=merged_df.drop('level_0', axis=1)

merged_df
clear_output()


In [ ]:
update_values = merged_df.sort_values(by='Course Name').values.tolist()
CourseWorksheet = sh.worksheet("Course A")
CourseWorksheet.update([merged_df.columns.values.tolist()] + update_values)
clear_output()